Starting small by scraping one page

In [3]:
import cloudscraper
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://www.beeradvocate.com/beer/profile/143/1352/" 
scraper = cloudscraper.create_scraper() 
info = scraper.get(url) 
 
print(info.status_code) 
 

200


In [177]:
soup = BeautifulSoup(info.text, 'lxml') 

In [178]:
name=soup.h1.find(string=True)
name

'Shiner Bock'

In [176]:
# soup.find('h1').text
##will give me both the beer title and brewery merged together

'Shiner BockSpoetzl Brewery'

In [74]:
soup.find('dl',{'class':'beerstats'}).text

'\nFrom:\nSpoetzl Brewery\n\xa0\nTexas, United States\nStyle:\nBockRanked #96\nABV:\n 4.5%\nScore:\n74Ranked #28,088\nAvg:\n3.21 | pDev: 19.63%\nReviews:\n1,234\nRatings:\n4,564\nStatus:\nActive\nRated:\n\nSunday at 10:11 AM\nAdded:\nOct 03, 1999\nWants:\n\xa0\xa0111\nGots:\n\xa0\xa01,369\n'

In [86]:
brewery=soup.find_all('a',attrs={'class':'Tooltip'})[4].text
brewery

'Spoetzl Brewery'

In [109]:
loaction=soup.find_all('dd', attrs={'class':'beerstats'})[1].a.text
loaction

'Texas'

In [149]:
country=soup.find_all('dd', attrs={'class':'beerstats'})[1].find_all('a')[1].text
country

'United States'

In [157]:
style=soup.find_all('a',attrs={'class':'Tooltip'})[5].text
style

'Bock'

In [174]:
ba_score=soup.find_all('span',attrs={'class':'Tooltip'})[5].text
ba_score
##official score on ba site 74::'okay'

'74'

In [160]:
over_all_rank=soup.find_all('a',attrs={'class':'Tooltip'})[7].text
over_all_rank
##over all rank is against all beers in ba site

'Ranked #28,088'

In [164]:
style_rank=soup.find_all('a',attrs={'class':'Tooltip'})[6].text
style_rank
## style rank is among all beers within style category

'Ranked #96'

In [169]:
alcohol_percent=soup.find_all('span',attrs={'class':'Tooltip'})[3].text
alcohol_percent

'4.5%'

In [181]:
avg_rating=soup.find_all('span',attrs={'class':'Tooltip'})[7].text
avg_rating
# Average across all ratings for this beer

'3.21'

In [185]:
percent_dev=soup.find_all('span',attrs={'class':'Tooltip'})[8].text
# Percentage of deviation within the ratings

'19.63%'

In [195]:
review_count=soup.find_all('span',attrs={'class':'Tooltip'})[10].text
review_count
# Number of reviews for this beer

'1,234'

In [196]:
rating_count=soup.find_all('span',attrs={'class':'Tooltip'})[12].text
rating_count
# Number of ratings for this beer

'4,564'

In [202]:
status=soup.find_all('span',attrs={'class':'Tooltip'})[14].text
status
# the beer's current activity status

'Active'

In [206]:
date_added=soup.find_all('span',attrs={'class':'Tooltip'})[18].text
date_added
# The date this beer was added to the site

'Oct 03, 1999'